In [10]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from scipy import sparse

In [2]:
book_df = pd.read_csv('data/Books.csv')
ratings_df = pd.read_csv('data/Ratings.csv')
user_df = pd.read_csv('data/Users.csv')
ratings_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
ratings_df = ratings_df[ratings_df['Book-Rating'] > 0]
ratings_df = ratings_df[ratings_df['User-ID'].isin(ratings_df['User-ID'].unique()[0:10000])]
ratings_df.shape

(61812, 5)

In [4]:
book_user_rating = book_df.merge(ratings_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)
book_user_rating.head()

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0002005018,Clara Callan,Richard Bruce Wright,8,5
1,0002005018,Clara Callan,Richard Bruce Wright,11676,8
2,0060973129,Decision in Normandy,Carlo D'Este,2954,8
3,0399135782,The Kitchen God's Wife,Amy Tan,11676,9
4,0399135782,The Kitchen God's Wife,Amy Tan,29526,9


In [5]:
d = {}
for i, j in enumerate(book_user_rating.ISBN.unique()):
    d[j] = i
book_user_rating["unique_id_book"] = book_user_rating["ISBN"].map(d)


test
s
Строим разреженную матрицу

In [6]:
df_pivot = book_user_rating.pivot(index='unique_id_book',
                                  columns='User-ID',
                                  values='Book-Rating').fillna(0)

In [7]:
df_pivot.head()

User-ID,8,9,12,14,16,17,19,22,26,32,...,278831,278832,278836,278843,278844,278846,278849,278851,278852,278854
unique_id_book,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_pivot.shape


(35582, 8709)

In [12]:
df_sparse = sparse.csr_matrix(df_pivot)
df_pivot.reset_index(inplace=True)

In [14]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(df_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [ ]:
def get_book_recommendation(ISBN, n_recommend: int):
    pass